In [ ]:
import os
import pandas as pd
from typing import List, Tuple

In [ ]:
flattened_solve_group_dir = ""
processed_data_population_pkl_path = ""

In [ ]:
all_matrices = [
    "conv_diff_1000", "af_0_k101", "af_shell9", "af23560", "airfoil_2d",
    "apache2", "appu", "atmosmodj", "BenElechi1", "bone010",
    "Bump_2911", "cage10", "cage13", "cage14", "chipcool1",
    "circuit_3", "coupled", "crankseg_1", "CurlCurl_2", "CurlCurl_4",
    "e40r0100", "ecology2", "epb1", "F1", "FEM_3D_thermal1",
    "FEM_3D_thermal2", "G3_circuit", "garon2", "hood", "inlet",
    "jan99jac040sc", "language", "marine1", "mc2depi", "memplus",
    "ns3Da", "parabolic_fem", "poisson3Da", "poisson3Db", "powersim",
    "pwtk", "rajat15", "rajat31", "shermanACb", "sme3Da",
    "stomach", "t2em", "thermal2", "tmt_unsym", "torso2",
    "torso3", "TSOPF_RS_b39_c7", "venkat01", "viscoplastic2", "wang3",
    "wang4", "Zhao1", "Zhao2"
]
dense_matrices = [
    "af23560", "airfoil_2d", "appu", "cage10", "chipcool1",
    "circuit_3", "coupled", "e40r0100", "epb1", "FEM_3D_thermal1",
    "garon2", "inlet", "jan99jac040sc", "memplus", "ns3Da",
    "poisson3Da", "powersim", "rajat15", "shermanACb", "sme3Da",
    "TSOPF_RS_b39_c7", "viscoplastic2", "wang3", "wang4", "Zhao1",
    "Zhao2"
]
ilu0_matrices = [
    "af23560", "airfoil_2d", "appu", "cage10", "chipcool1",
    "epb1", "FEM_3D_thermal1", "inlet", "memplus", "poisson3Da",
    "powersim", "sme3Da", "viscoplastic2", "wang3", "wang4",
    "Zhao1", "Zhao2"
]

In [ ]:
setup_ids = ["ilu0", "ilutp_1em2", "ilutp_1em4", "unpreconditioned_dense", "unpreconditioned"]
inner_iterations = [10, 20, 30, 40, 50, 100, 150, 200]
n_iterations = 3
solvers = [
    "FP16", "FP32", "FP64",
    "SD_OuterRestartCount", "SD_CheckStagnation", "SD_RelativeResidualThreshold",
    "OuterRestartCount", "CheckStagnation", "RelativeResidualThreshold", "ThresholdToStagnation"
]
setup_id_matrices_mapping = {
    "ilu0": ilu0_matrices,
    "ilutp_1em2": dense_matrices,
    "ilutp_1em4": dense_matrices,
    "unpreconditioned_dense": dense_matrices,
    "unpreconditioned": all_matrices
}

In [ ]:
# Track population on matrices than flattened setup-inner-iterations-and-solvers
def check_matrix_solver_inner_iter_populated_and_get_paths(
    solve_group_dir_path: str, matrix: str, solver: str, inner_iter: int, n_iterations: int
) -> Tuple[bool, List[str]]:
    file_paths = []
    for iteration in range(n_iterations):
        file_path = os.path.join(solve_group_dir_path, matrix, str(iteration), solver+".json")
        file_paths.append(file_path)
        if not os.path.exists(file_path):
            return (False, [])
    return (True, file_paths)

data = []
for setup in setup_ids:
    for matrix in setup_id_matrices_mapping[setup]:
        for solver in solvers:
            for inner_iter in inner_iterations:
                solve_group_dir_path = os.path.join(
                    flattened_solve_group_dir,
                    f"{setup}_inner_iter_{inner_iter}"
                )
                populated, file_paths = check_matrix_solver_inner_iter_populated_and_get_paths(
                    solve_group_dir_path, matrix, solver, inner_iter, n_iterations
                )
                data.append({
                    "setup": setup,
                    "matrix": matrix,
                    "solver": solver,
                    "inner_iter": inner_iter,
                    "populated": populated,
                    "file_paths": file_paths
                })

In [ ]:
data_population = pd.DataFrame(data, columns=["setup", "matrix", "solver", "inner_iter", "populated", "file_paths"])
data_population.set_index(["setup", "matrix", "solver", "inner_iter"])
data_population.to_pickle(processed_data_population_pkl_path)
display(data_population)

In [ ]:
missing_data_population = data_population.loc[~data_population["populated"]]
display(missing_data_population)